In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import sqlite3
import os
import numpy as np
from requests_html import HTMLSession
from requests_file import FileAdapter

session = HTMLSession()

session.mount('file://',FileAdapter())
pwd = os.getcwd().replace("\\","/")
html_obj = session.get(f'file:///{pwd}/台股公告 _ Anue鉅亨.html')
soup = BeautifulSoup(html_obj.text,"html.parser")

In [2]:
titletime = []
id = []
idBefore = []
content = []
  

# # 今日新闻
inputdate = '2022-09-20'


# # 日期============================
times = soup.select('._67tN')
for time in times:
  head,mid,tail = time.find('time')['datetime'].partition('T') # partition‘T'之後分為頭部，中部，尾部三個
  titletime.append(head)
# print(len(titletime))

# 內容===============================================================================================================
titles = soup.select('._1xc2')
# 1.去除特殊字符
for title in titles:
  head,mid,tail = title.find('h3').text.partition(':')
  if head[0] == '「':
    head2 = head.replace('「','') # replace（代替），將字符替換成
    head3 = head2.find('、') # 在第幾位

    if head3 < 0: # 沒有、號
      head2 = head2.split('」',1) # split（切割），切割為【」以前的內容】和【」以後的內容】
      head2 = head2[0]
      idBefore.append(head2)
    else: # 有、號
      head2 = head2[:head3]
      head2 = head2.split('」',1) 
      head2 = head2[0]
      idBefore.append(head2)
  else:
    idBefore.append(head)
    titleText = title.text
    titleText = titleText.strip()
  content.append(titleText)
# 2.去除【月營收】
for x in idBefore:
  x2 = x.find('月營收')
  if x2 < 0: # 沒有【月營收】
    id.append(x)
  else: # 有【月營收】
    x3 = x2 - 1
    x4 = x[:x3]
    id.append(x4)
# =================================================================================================================


data = {
  'date': titletime,
  'code': id,
  'content': content
}

df = pd.DataFrame(data)

# 去除特殊字符 ===================================================
for x in range(len(df['content'])):
  df.iloc[x]['content'] = df.iloc[x]['content'].replace('','')

# ===============================================================


newsToday = pd.Timestamp.today().strftime("%Y-%m-%d") # 格式化日期
df_mask = df['date']== inputdate            # 日期為今天的存檔
positions = np.flatnonzero(df_mask)
filtered_df=df.iloc[positions]
# print(filtered_df)


# connecting to sqlite
conn = sqlite3.connect('anuenews.db')
# cursor object
cursor = conn.cursor()
uniqueSql = '''select * from news t'''


for index, row in filtered_df.iterrows():
  try:
    cursor.execute(
        """INSERT OR IGNORE INTO news 
          (date, code, content)
          values(?,?,?)""",
          (row['date'],
          row['code'],
          row['content']),
        )
    conn.commit()
  except:
    pass

# print(filtered_df)

# 確認當日新聞 ========================================================================
dfToday = pd.read_sql(uniqueSql,conn)
# df_maskToday = dfToday['date']==newsToday            # 日期為今天的存檔
df_maskToday = dfToday['date']==inputdate           # 日期為今天的存檔
positionsToday = np.flatnonzero(df_maskToday)
filtered_dfToday=dfToday.iloc[positionsToday]
# ======================================================================================

filtered_dfToday

,date,code,content
4439,2022-09-20,宏正,宏正:本公司受邀參加臺灣證券交易所及寬量國際共同主辦之線上法說會「12th Taiwan C...
4440,2022-09-20,安國,安國:安國國際科技股份有限公司（公司代號：8054）註銷收回股票事宜。
4441,2022-09-20,中和,中和:雋揚國際股份有限公司（原公司名稱：中和羊毛工業股份有限公司，公司代號：1439）因公司...
4442,2022-09-20,昶昕,「昶昕」自 2022.09.20起得為融資融券交易
4443,2022-09-20,華義,華義:本公司盈餘轉增資新股發放及上櫃日期公告
...,...,...,...
4521,2022-09-20,和進,和進:(更正)依證券商營業處所買賣有價證券業務規則公告本公司股票調整變更交易方法之事由，惟仍...
4522,2022-09-20,加百裕,加百裕:公告本公司111年08月份自結合併報表之負債比率、流動比率及速動比率
4523,2022-09-20,臻鼎-KY,臻鼎-KY:本公司受邀參加台灣證券交易所與寬量國際共同舉辦之線上法說會「12th Taiwa...
4524,2022-09-20,訊聯,訊聯:公告本公司GTP實驗室之細胞製備場所，依據「特定醫療技術檢查檢驗醫療儀器施行或使用管理...


In [1]:
# 创建资料库 ====================================================================
# import sqlite3


# # connecting to sqlite
# conn = sqlite3.connect('anuenews.db')
# # cursor object
# cursor = conn.cursor()

# # Creating table
# news = """ CREATE TABLE news (
#     date DATE,
#     code TEXT,
#     content TEXT UNIQUE, 
#     UNIQUE(content)
# ); """

# cursor.execute(news)
# ================================================================================